# Text Generation

Mainly copied and adapted from the Text Generation RNN example

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [2]:
#import dataset
path_to_file = "oneliners.tsv"


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 73485 characters


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

98 unique characters


## Text Vectorisation

In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [6]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

﻿
Y
o
u
 


In [7]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

"\ufeffYou can't spell advertisements without semen between the tits.\nI was gonna tell a time travelling jo"
"ke but you guys didn't like it\nMy three favorite things are eating my family and not using commas.\nTh"
'e share button on reddit should be called spreddit\nIf my wife made a dollar for every sexist joke I m'
"ake, she'd be $0.77 richer right now.\nI wanted to marry my English teacher when she got out of jail, "
'but apparently, you can’t end a sentence with a proposition.\nI saw a double-entendre contest, so I en'


In [8]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

## Training Batches

In [9]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build Model

In [10]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [11]:
if tf.test.is_gpu_available(): #GPU
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    # tack on two RNN layers 
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
      
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [13]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

## Try Model

In [14]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 98) # (batch_size, sequence_length, vocab_size)


In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           25088     
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (64, None, 1024)          6297600   
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (64, None, 1024)          6297600   
_________________________________________________________________
dense (Dense)                (64, None, 98)            100450    
Total params: 16,659,042
Trainable params: 16,659,042
Non-trainable params: 0
_________________________________________________________________


## Train

In [16]:
def loss(labels, logits):
#   return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 98)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.58471


In [17]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

Checkpoints

In [18]:
# Directory where the checkpoints will be saved
checkpoint_dir = './text100'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [19]:
EPOCHS=500

In [20]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/500
11/11 [==============================] - 5s 436ms/step - loss: 3.9734
Epoch 2/500
11/11 [==============================] - 2s 211ms/step - loss: 3.2738
Epoch 3/500
11/11 [==============================] - 4s 365ms/step - loss: 3.1005
Epoch 4/500
11/11 [==============================] - 3s 247ms/step - loss: 2.9369
Epoch 5/500
11/11 [==============================] - 4s 400ms/step - loss: 2.7292
Epoch 6/500
11/11 [==============================] - 4s 395ms/step - loss: 2.5290
Epoch 7/500
11/11 [==============================] - 3s 236ms/step - loss: 2.4173
Epoch 8/500
11/11 [==============================] - 3s 240ms/step - loss: 2.3408
Epoch 9/500
11/11 [==============================] - 3s 300ms/step - loss: 2.2637
Epoch 10/500
11/11 [==============================] - 3s 292ms/step - loss: 2.1915
Epoch 11/500
11/11 [==============================] - 3s 289ms/step - loss: 2.1196
Epoch 12/500
11/11 [==============================] - 2s 195ms/step - loss: 2.0442
Epoch 13/500


11/11 [==============================] - 3s 246ms/step - loss: 0.0783
Epoch 197/500
11/11 [==============================] - 2s 207ms/step - loss: 0.0787
Epoch 198/500
11/11 [==============================] - 4s 319ms/step - loss: 0.0763
Epoch 199/500
11/11 [==============================] - 4s 386ms/step - loss: 0.0768
Epoch 200/500
11/11 [==============================] - 3s 262ms/step - loss: 0.0761
Epoch 201/500
11/11 [==============================] - 3s 236ms/step - loss: 0.0782
Epoch 202/500
11/11 [==============================] - 4s 324ms/step - loss: 0.0748
Epoch 203/500
11/11 [==============================] - 4s 339ms/step - loss: 0.0761
Epoch 204/500
11/11 [==============================] - 4s 335ms/step - loss: 0.0778
Epoch 205/500
11/11 [==============================] - 2s 212ms/step - loss: 0.0770
Epoch 206/500
11/11 [==============================] - 4s 400ms/step - loss: 0.0779
Epoch 207/500
11/11 [==============================] - 3s 281ms/step - loss: 0.0753
Epoch 

11/11 [==============================] - 5s 423ms/step - loss: 0.0675
Epoch 391/500
11/11 [==============================] - 2s 218ms/step - loss: 0.0653
Epoch 392/500
11/11 [==============================] - 3s 292ms/step - loss: 0.0676
Epoch 393/500
11/11 [==============================] - 3s 262ms/step - loss: 0.0689
Epoch 394/500
11/11 [==============================] - 6s 500ms/step - loss: 0.0694
Epoch 395/500
11/11 [==============================] - 3s 268ms/step - loss: 0.0692
Epoch 396/500
11/11 [==============================] - 3s 317ms/step - loss: 0.0679
Epoch 397/500
11/11 [==============================] - 4s 384ms/step - loss: 0.0684
Epoch 398/500
11/11 [==============================] - 5s 474ms/step - loss: 0.0669
Epoch 399/500
11/11 [==============================] - 3s 254ms/step - loss: 0.0662
Epoch 400/500
11/11 [==============================] - 3s 314ms/step - loss: 0.0672
Epoch 401/500
11/11 [==============================] - 3s 290ms/step - loss: 0.0656
Epoch 

## Generate Oneliners!

In [20]:
tf.train.latest_checkpoint(checkpoint_dir)

'./text100/ckpt_500'

In [21]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1) 

#model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.load_weights('./text100/ckpt_250')


model.build(tf.TensorShape([1, None]))

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            25088     
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
cu_dnngru_4 (CuDNNGRU)       (1, None, 1024)           6297600   
_________________________________________________________________
cu_dnngru_5 (CuDNNGRU)       (1, None, 1024)           6297600   
_________________________________________________________________
dense_1 (Dense)              (1, None, 98)             100450    
Total params: 16,659,042
Trainable params: 16,659,042
Non-trainable params: 0
_________________________________________________________________


In [37]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 4500

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  temperature = 1.05

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [38]:
oneliners = generate_text(model, start_string=u"I")
print(oneliners)


If a fire-fighter's business can go up in smoke, and a plumber's business can go down the drain, can th people who longer dipstick for my car, mine doesn't return to you it was probably a stick.
...but you can halve your cake and eat it in tw people who live in Seville are referred to as Sevilians.
I don't trust new cafes, because they fill stars and thought to myself, “I can’t turn that down!"""
I just got back from China; I feed bet shop today, and was shocked to find out he all really weird because they only do their job after they’re fired.
Wine doesn't me winning computer store to brink be all the word “disguise” in disguise
If lazinesswer collapsed, I shop today, and was shocked to find out how much all my wife’s vibrators cost, I mean, she’s sitting on me.
A bad analogy is like a bad analogy
English puns make me feet.
I totally understand how batteries feel because I'm rarely included in things either.
I asks family.
A cop just knocked on my door and told me that my dogs were ch